## TMA4212 Project 1
#### Group: Nikolai Rasmus Sætren, Thomas Olaussen, Tiago Alexandre Alcobia Pereira


#### Part 1

In [ ]:
%matplotlib inline
import numpy as np
import scipy as sp
from numpy import sin,cos
import pandas as pd #To format the output of the notebook.  
from IPython.display import display
import time
import matplotlib.pyplot as plt
newparams = {'figure.figsize': (8.0, 4.0), 'axes.grid': True,
             'lines.markersize': 8, 'lines.linewidth': 2,
             'font.size': 14}
from matplotlib import cm
plt.rcParams.update(newparams)

In [ ]:
class BVP(object): 
    def __init__(self, g=0, f=0, a = 0, r = 1):
        self.g = g         # Boundary condition
        self.f = f         # right hand side 
        self.a = a         # a
        self.r = r

$M \times M$ grid

In [ ]:
def blockTridiag(L,D,U,M,N=0, eta = 1):
    ''' 
    Returns sparse (M*M x M*M) block tridiagonal matrix with lower entries L, diagonal entries D, and upper entries U.
    L,D,U are (M x M) matrices.
    '''
    if N ==0:
        N = M
    
    A = sp.sparse.bmat([[D if i == j else  L if i-j==-1 else U if i-j ==1 else None for i in range(N)] for j in range(N)], format = 'csr')
    return A


def tridiag(l, d, u, M):
    '''
    Returns sparse (M x M) tridiagonal matrix with lower entries l, diagonal entries d, and upper entries u
    '''
    e = np.ones(M)
    diagonals = [l*e[:-1],d*e,u*e[:-1]]       
    A = sp.sparse.diags(diagonals,[-1,0,1])
    return A

def plot_solution(x, y, U, txt='Solution'):
    # Plot the solution of the heat equation
    
    fig, ax = plt.subplots(subplot_kw={"projection": "3d"})
    X, Y = np.meshgrid(x,y)
    ax.plot_surface(X, Y, U.T, cmap=cm.coolwarm)
    ax.set_zlabel('                              ')
    plt.xlabel('x')
    plt.ylabel('y')
    plt.title(txt)
    plt.show()

In [ ]:

def solve_square_grid(BVP,M):
    '''
    Solves the BVP numerically using a square grid, when r = 1
    '''
    
    h = 1/(M+1)

    # Creating the matrix
    a = BVP.a
    g = BVP.g
    f = BVP.f
    L = tridiag(-1,0,0,M)
    D = tridiag(-a,2*(1+a),-a,M)
    U = tridiag(0,0,-1,M)
    A = blockTridiag(L,D,U,M)

    # Define the grid
    U = np.zeros(M*M)
    T = np.zeros((M+2,M+2))
    x = np.linspace(0,1,M+2)
    y = np.linspace(0,1,M+2)

    T[0,:]  =  g(0,y)
    T[-1,:] =  g(1,y)
    T[:,0]  =  g(x,0)
    T[:,-1] =  g(x,1)

    # Boundary conditions
    G = np.zeros((M,M))
    G = a*T[:-2,1:-1] + a*T[2:,1:-1] + T[:-2,:-2] + T[2:,2:]
    G = G.flatten(order = 'F')

    # Right hand side:
    F = np.zeros((M,M))
    yv,xv = np.meshgrid(y[1:-1],x[1:-1])

    F = f(xv,yv)
    F = F.flatten(order='F')

    U = sp.sparse.linalg.spsolve(A,h**2*F+G)

    # Order the results into a 2D array
    T[1:-1,1:-1] = U.reshape((M,M),order='F')
    
    return T,x,y




In [ ]:
a = 1

def g0(x,y):
    return x*y*cos(x*y)
def f0(x,y):
    return 20*cos(5*x-2*y)*sin(4*x+7*y)

def g1(x,y):
    return cos(x)*sin(2*y)

def f1(x,y):
    return (a+1+4)*cos(x)*sin(2*y)+4*sin(x)*cos(2*y)

ex0 = BVP(g0,f0,a=a)
ex1 = BVP(g1,f1,a=a)
T0,x0,y0 = solve_square_grid(ex0,100)
T,x,y = solve_square_grid(ex1,100)

xv,yv = np.meshgrid(x,y)

plot_solution(x0,y0,T0,'Numerical Solution')

plot_solution(x,y,T,"Numerical solution")

plot_solution(x,y,g1(xv,yv).T, "Analytical solution")

plot_solution(x,y,g1(xv,yv).T-T, "Error")

print(np.max(np.abs(g1(xv,yv).T-T)))



#### **c)** Error

In [ ]:
def conv_error(BVP, Ms, method):
    '''
    Returns error, h values and order of a method for a given boundary value problem
    '''
    E = np.zeros(len(Ms))        # Error
    H = np.zeros(len(Ms))        # h values

    for i,M in enumerate(Ms):
        T,x,y = method(BVP,M)
        xv,yv = np.meshgrid(x,y)
        E[i] =np.nanmax(np.abs(g1(xv,yv)-T.T))
        H[i] = 1/(M+1)

    order = 0
    try:
        order = np.polyfit(np.log(H), np.log(E),1)[0] # Order of convergence
    except:
        pass
    
    return E, H, order

def conv_table(E,H):
    Rate=np.zeros(np.size(E))
    Rate[1:]=np.log10(E[1:]/E[:-1])/np.log10(H[1:]/H[:-1])
    pd.options.display.float_format = '{:.8f}'.format
    df = pd.DataFrame(data={'h': H, 'Error': E ,'Rate':Rate}) 
    display(df)


In [ ]:
Ms = [40*(2*i+1) for i in range(5)]

E,H, order = conv_error(ex1,Ms,solve_square_grid)
    


In [ ]:
conv_table(E,H)

In [ ]:
plt.figure()
plt.loglog(H,E,label=f'$p$ = {order:.3f}')
plt.title(f'Square grid $a = {a}$')
plt.xlabel('$h$')
plt.ylabel('Error')
plt.legend()
plt.show()

#### **d)**

Since $r$ is irrational we can ignore the $r = 0$ case

In [ ]:
def solve_rectangular_grid(BVP, M):
    '''
    Numerically solves the FDM on a rectangular grid.
    
    '''
    
    a = BVP.a
    g = BVP.g
    f = BVP.f
    r = BVP.r
    
    
    # Define the grid
    h = 1/(M+1)
    k = np.abs(r)*h
    N = int(np.floor(1/k))

    U = np.zeros(M*N)
    T = np.zeros((M+2,N+2))
    x = np.linspace(0,1,M+2)
    y = np.ones(N+2)
    y[:-1] = np.arange(0,1,k)

    eta = (1-y[-2])/k

    # Creating the matrix
    if r >= 0:
        L = tridiag(-1,0,0,M)
        D = tridiag(-a,2*(1+a),-a,M)
        U = tridiag(0,0,-1,M)
        D2 = tridiag(-a,2*(a+1/eta),-a,M)
        L2 = tridiag(-2/(eta+1),0,0,M)
        A = blockTridiag(L,D,U,M,N)
        A[M*(N-1):,M*(N-2):M*(N-1)] = L2
        A[M*(N-1):,M*(N-1):] = D2

    else:
        L = tridiag(0,0,-1,M)
        D = tridiag(-a,2*(1+a),-a,M)
        U = tridiag(-1,0,0,M)
        D2 = tridiag(-a,2*(a+1/eta),-a,M)
        L2 = tridiag(0,0,-2/(eta+1),M)
        A = blockTridiag(L,D,U,M,N)
        A[M*(N-1):,M*(N-2):M*(N-1)] = L2
        A[M*(N-1):,M*(N-1):] = D2
        
        

    T[0,:]  =  g(0,y)
    T[-1,:] =  g(1,y)
    T[:,0]  =  g(x,0)
    T[:,-1] =  g(x,1)

    # Boundary conditions
    if r > 0:
        T1 = T.copy()
        T1[:,-1] = 0
        T1[0,-3] *= 2/(eta + 1)
        G = np.zeros((M,N))
        G = a*T[:-2,1:-1] + a*T[2:,1:-1] + T1[:-2,:-2] + T1[2:,2:]
        G[:,-1] += 2/(eta*(eta+1))*g(x+eta*h,1)[1:-1]
        G = G.flatten(order = 'F')


    
    else:
        T1 = T.copy()
        T1[:,-1] = 0
        T1[-1,-3] *= 2/(eta+1)
        G = np.zeros((M,N))
        G = a*T[:-2,1:-1] + a*T[2:,1:-1] + T1[2:,:-2] + T1[:-2,2:]
        G[:,-1] += 2/(eta*(eta+1))*g(x-eta*h,1)[1:-1]
        G = G.flatten(order = 'F')


    # Right hand side:
    F = np.zeros((M,N))
    yv,xv = np.meshgrid(y[1:-1],x[1:-1])

    F = f(xv,yv)
    F = F.flatten(order='F')

    U = sp.sparse.linalg.spsolve(A,h**2*F+G)

    # Order the results into a 2D array
    T[1:-1,1:-1] = U.reshape((M,N),order='F')


    return T,x,y





Convergence

In [ ]:
r = np.sqrt(2)
a = 1

def g2(x,y):
    return cos(x)*sin(2*y)
def f2(x,y):
    return (a+1+4*r**2)*cos(x)*sin(2*y)+4*r*sin(x)*cos(2*y)


ex2 = BVP(g2,f2,a,r)

T,x,y = solve_rectangular_grid(ex2,100)

xv,yv = np.meshgrid(x,y)

plot_solution(x,y,T,"Numerical solution")

plot_solution(x,y,g2(xv,yv).T, "Analytical solution")

plot_solution(x,y,g2(xv,yv).T-T, "Error")

print(np.max(np.abs(g2(xv,yv).T-T)))


In [ ]:
Ms = [40*(2*i+1) for i in range(5)]
E,H,order = conv_error(ex2,Ms,solve_rectangular_grid)

In [ ]:
conv_table(E,H)

In [ ]:
plt.figure()
plt.loglog(H,E,label=f'$p$ = {order:.3f}')
plt.title(f'Rectangular grid, $a = {a}, r = {r:.3f}$')
plt.xlabel('$h$')
plt.ylabel('Error')
plt.legend()
plt.show()

In [ ]:
def g_interesting(x,y):
    return (1-x)*y+sin(y)/4
def f_interesting(x,y):
    return 30*(sin((4*x)**2)+cos(3*y))

ex_interesting = BVP(g_interesting,f_interesting,np.sqrt(2),1/3*np.pi)

T,x,y = solve_rectangular_grid(ex_interesting,200)

plot_solution(x,y,T,'Numerical solution')

### Problem 2

In [ ]:
def gamma(x):
    return 1/2*(cos(np.pi*x)+1)

def index2d(indx, M, shift= 0):
    x = indx%M + 1 
    y = indx//M + 1
    A = np.array([x,y])
    A = (A.T+shift).T
    return tuple(A)

def g3(x,y):
    return cos(x)*sin(2*y)

def f3(x,y):
    return 5*cos(x)*sin(2*y)

ex3 = BVP(g3,f3)



def irregular_fattening(BVP,M,gamma = gamma):
    '''
    Numerical method in [0,1]^2 with additional irregular upper boundary. 
    Fattens the boundary by adding additional grid points outside it.

    gamma: boundary function
    '''

    g = BVP.g
    f = BVP.f
    L = tridiag(0,-1,0,M)
    D = tridiag(-1,4,-1,M)
    U = tridiag(0,-1,0,M)
    A = blockTridiag(L,D,U,M)

    # [0,1]x[0,1] grid
    h = 1/(M+1)
    x = np.linspace(0,1,M+2)
    y = np.linspace(0,1,M+2)
    yv,xv = np.meshgrid(y,x)
    T = np.zeros((M+2,M+2))


    # Points inside the grid
    ubnd = gamma(x)  # upper boundary values
    boundary = (yv.T == ubnd).T
    boundary[:,0] = 1
    boundary[0,:] = 1
    grid_cand = (yv.T <= ubnd).T.astype(int)
    interior =  (grid_cand - boundary)


    # Remove matrix entries for points outside the boundary
    indx_keep = np.where(interior[1:-1,1:-1].flatten(order='F') == 1)[0]
    A = A[indx_keep]
    A = A.T[indx_keep].T

    # Categorising boundary adjacent points
    exterior = (interior == 0).astype(int)
    W_bndry=np.where((interior[1:-1,1:-1]+exterior[:-2,1:-1]).flatten(order='F')==2)[0]
    E_bndry=np.where((interior[1:-1,1:-1]+exterior[2:,1:-1]).flatten(order='F')==2)[0]
    N_bndry=np.where((interior[1:-1,1:-1]+exterior[1:-1,2:]).flatten(order='F')==2)[0]
    S_bndry=np.where((interior[1:-1,1:-1]+exterior[1:-1,:-2]).flatten(order='F')==2)[0]

    # Boundary conditions
    G = np.zeros(M*M)
    P_N = np.array([0,1])
    P_S = np.array([0,-1])
    P_E = np.array([1,0])
    P_W = np.array([-1,0])

    G[N_bndry] += g(xv,yv)[index2d(N_bndry,M,P_N)]
    G[S_bndry] += g(xv,yv)[index2d(S_bndry,M,P_S)]
    G[W_bndry] += g(xv,yv)[index2d(W_bndry,M,P_W)]
    G[E_bndry] += g(xv,yv)[index2d(E_bndry,M,P_E)]

    G = G[indx_keep]

    # RHS
    F = f(xv[1:-1,1:-1],yv[1:-1,1:-1]).flatten(order='F')[indx_keep]

    # Solve
    exterior_indx = np.where((grid_cand==0))
    T[exterior_indx] = None
    boundary_indx = np.where(boundary==1)
    T[boundary_indx] = g(xv,yv)[boundary_indx]


    U = sp.sparse.linalg.spsolve(A,h**2*F+G)

    T[index2d(indx_keep,M)] = U
    
    return T,x,y

T,x,y = irregular_fattening(ex3,100,gamma)


In [ ]:
plot_solution(x,y,T, 'Numerical solution')
yv,xv = np.meshgrid(y,x)
T_anal = g3(xv,yv)
plot_solution(x,y,(T_anal+T-T), 'Analytic solution')       # Multiplying with NaN to get corresponding NaN in T_anal
plot_solution(x,y,(T_anal-T), 'Error')

print(np.nanmax(np.abs(T_anal-T)))


In [ ]:
Ms = [40*(1+2*i) for i in range(5)]
E,H,order =conv_error(ex3,Ms,irregular_fattening)

In [ ]:
conv_table(E,H)

In [ ]:
plt.figure()
plt.loglog(H,E,label=f'$p$ = {order:.3f}')
plt.title('Irregular boundary with fattening')
plt.xlabel('$h$')
plt.ylabel('Error')
plt.legend()
plt.show()

Method 2: Adding nodes

In [ ]:
def gamma(x):
    return 1/2*(cos(np.pi*x)+1)

def gamma_inv(y):
    return np.arccos(2*y-1)/np.pi

def g3(x,y):
    return cos(x)*sin(2*y)

def f3(x,y):
    return 5*cos(x)*sin(2*y)

ex3 = BVP(g3,f3)



def irregular_adjusting(BVP,M,gamma = gamma, gamma_inv=gamma_inv):
    '''
    Numerical method in [0,1]^2 with additional irregular upper boundary.
    Adjusts the discretisation for points adjacent to the boundary 

    gamma: boundary function
    '''

    g = BVP.g
    f = BVP.f
    L = tridiag(0,-1,0,M)
    D = tridiag(-1,4,-1,M)
    U = tridiag(0,-1,0,M)
    A = blockTridiag(L,D,U,M)

    # [0,1]x[0,1] grid
    h = 1/(M+1)
    x = np.linspace(0,1,M+2)
    y = np.linspace(0,1,M+2)
    yv,xv = np.meshgrid(y,x)
    T = np.zeros((M+2,M+2))


    # Points inside the grid
    ubnd = gamma(x)  # upper boundary values
    boundary = (yv.T == ubnd).T
    boundary[:,0] = 1
    boundary[0,:] = 1
    grid_cand = (yv.T <= ubnd).T.astype(int)
    interior =  (grid_cand - boundary)


    # Categorising boundary adjacent points
    exterior = (interior == 0).astype(int)
    N_bndry=np.where((interior[1:-1,1:-1]+exterior[1:-1,2:]).flatten(order='F')==2)[0]
    S_bndry=np.where((interior[1:-1,1:-1]+exterior[1:-1,:-2]).flatten(order='F')==2)[0]
    E_bndry=np.where((interior[1:-1,1:-1]+exterior[2:,1:-1]).flatten(order='F')==2)[0]
    W_bndry=np.where((interior[1:-1,1:-1]+exterior[:-2,1:-1]).flatten(order='F')==2)[0]

    # Boundary conditions
    G = np.zeros(M*M)
    P_N = np.array([0,1])
    P_S = np.array([0,-1])
    P_E = np.array([1,0])
    P_W = np.array([-1,0])


    Nx = x[index2d(N_bndry,M)[0]]                          # x-vals for the north boundary adjacent points
    Ey = y[index2d(E_bndry,M)[1]]                          # y-vals for the east boundary adjacent points

    etaN = (gamma(Nx) - yv[index2d(N_bndry,M)])/h
    etaE = (gamma_inv(Ey) - xv[index2d(E_bndry,M)])/h

    NS_bndry = np.intersect1d(N_bndry,S_bndry)
    EW_bndry = np.intersect1d(E_bndry,W_bndry)
    NE_bndry = np.intersect1d(N_bndry,E_bndry)

    G[N_bndry] += 2/(etaN*(etaN+1))*g(Nx,gamma(Nx))         
    G[E_bndry] += 2/(etaE*(etaE+1))*g(gamma_inv(Ey),Ey)

    G[S_bndry] += g(xv,yv)[index2d(S_bndry,M,P_S)]        
    G[W_bndry] += g(xv,yv)[index2d(W_bndry,M,P_W)]


    NS_indx = np.where(np.in1d(N_bndry,NS_bndry))[0]
    EW_indx = np.where(np.in1d(E_bndry,EW_bndry))[0]
    NE_indx = np.where(np.in1d(N_bndry,E_bndry))[0]
    EN_indx = np.where(np.in1d(E_bndry,N_bndry))[0]

    N_strict = np.setdiff1d(N_bndry,NS_bndry)                   # strictly north adjacent points
    E_strict = np.setdiff1d(E_bndry,EW_bndry)                   # strictly east adjacent points
    N_strict_indx = np.where(np.in1d(N_bndry,N_strict))[0]
    E_strict_indx = np.where(np.in1d(E_bndry,E_strict))[0]


    G[NS_bndry] += (2/(etaN[NS_indx]+1)-1)*g(xv,yv)[index2d(NS_bndry,M,P_S)]                     
    G[EW_bndry] += (2/(etaE[EW_indx]+1)-1)*g(xv,yv)[index2d(EW_bndry,M,P_W)]                           

    A[N_bndry,N_bndry] = 0
    A[E_bndry,E_bndry] = 0

    A[N_bndry,N_bndry] = 2+2/etaN
    A[E_bndry,E_bndry] = 2/etaE + 2

    A[N_strict,N_strict-M] = -2/(etaN[N_strict_indx]+1) 
    A[E_strict,E_strict-1] = -2/(etaE[E_strict_indx]+1)

    # Points that are both north and east boundary adjacent
    A[NE_bndry,NE_bndry] = 0
    A[NE_bndry,NE_bndry] = 2/etaN[NE_indx] + 2/etaE[EN_indx]
    
    # Remove matrix entries for points outside the boundary
    indx_keep = np.where(interior[1:-1,1:-1].flatten(order='F') == 1)[0]

    A = A[indx_keep]        
    A = A.T[indx_keep].T
    
    G = G[indx_keep]

    # RHS
    F = f(xv[1:-1,1:-1],yv[1:-1,1:-1]).flatten(order='F')[indx_keep]

    # Solve
    exterior_indx = np.where((grid_cand==0))
    T[exterior_indx] = None
    boundary_indx = np.where(boundary==1)
    T[boundary_indx] = g(xv,yv)[boundary_indx]
    
    U = sp.sparse.linalg.spsolve(A,h**2*F+G)

    T[index2d(indx_keep,M)] = U

    return T,x,y

T,x,y = irregular_adjusting(ex3,500,gamma)

In [ ]:
plot_solution(x,y,T, 'Numerical solution')
yv,xv = np.meshgrid(y,x)
T_anal = g3(xv,yv)

plot_solution(x,y,T_anal+T-T, 'Analytic solution')       # Multiplying with NaN to get corresponding NaN in T_anal
plot_solution(x,y,(T_anal-T), 'Error')

print(np.nanmax(np.abs(T_anal-T)))

In [ ]:
Ms = [40*(1+2*i) for i in range(5)]
E,H,order =conv_error(ex3,Ms,irregular_adjusting)

In [ ]:
conv_table(E,H)

In [ ]:
plt.figure()
plt.loglog(H,E,label=f'$p$ = {order:.3f}')
plt.title('Irregular boundary with added boundary points')
plt.xlabel('$h$')
plt.ylabel('Error')
plt.legend()
plt.show()

In [ ]:
t0 = time.time()
for i in range(20):
    irregular_adjusting(ex3,300)
t1 = time.time()
for i in range(20):
    irregular_fattening(ex3,300)
t2 = time.time()

dt1 = t1-t0
dt2 = t2-t1

print(f'Modified discretisation: {dt1}')
print(f'Fattening the boundary: {dt2}')